In [6]:
import csv
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os

print(tf.__version__)

# gpus = tf.config.list_physical_devices('GPU')
# if gpus:
#   try:
#     # Currently, memory growth needs to be the same across GPUs
#     for gpu in gpus:
#       tf.config.experimental.set_memory_growth(gpu, True)
#     logical_gpus = tf.config.list_logical_devices('GPU')
#     print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
#   except RuntimeError as e:
#     # Memory growth must be set before GPUs have been initialized
#     print(e)

2.16.2


In [32]:
imageFileNames = tf.data.Dataset.list_files('./images/*', shuffle=True)

labelDict = {}
with open("./dev_data_2025.csv", "r") as f:
    for imgLabel in csv.DictReader(f):
        labelDict[imgLabel['imageID']] = [
            imgLabel['label'], 
            imgLabel['cell_shape'], 
            imgLabel['nucleus_shape'], 
            imgLabel['cytoplasm_vacuole']
        ]

In [40]:
imageFileNames.take(10).map(lambda x: tf.py_function(lambda y: print(y.numpy()), [x], [tf.uint8]))

<_MapDataset element_spec=(TensorSpec(shape=<unknown>, dtype=tf.uint8, name=None),)>

In [53]:
# Making the datasets


def process_path(filePath):
    print(filePath)
    label = labelDict[filePath.split(os.path.sep)[-1][:9]]
    # Load the raw data from the file as a string
    img = tf.io.read_file(filePath)
    img_decode = tf.io.decode_jpeg(img, channels=3)
    return img_decode, label


valSize = int(imageFileNames.cardinality().numpy() * 0.2)

trainData = imageFileNames \
  .skip(valSize) \
  .map(lambda x: tf.py_function(process_path, num_parallel_calls=tf.data.AUTOTUNE)
valData = imageFileNames \
  .take(valSize) \
  .map(process_path, num_parallel_calls=tf.data.AUTOTUNE)

print(tf.data.experimental.cardinality(trainData).numpy())
print(tf.data.experimental.cardinality(valData).numpy())

for image, label in trainData.take(1):
  print("Image shape: ", image.numpy().shape)
  print("Label: ", label.numpy())

Tensor("args_0:0", shape=(), dtype=string)


AttributeError: in user code:

    File "/tmp/ipykernel_8842/2854129465.py", line 6, in process_path  *
        label = labelDict[filePath.split(os.path.sep)[-1][:9]]

    AttributeError: 'SymbolicTensor' object has no attribute 'split'


In [45]:
# Visuals

image_batch, label_batch = next(iter(trainData))

plt.figure(figsize=(10, 10))
for i in range(9):
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow(image_batch[i].numpy().astype("uint8"))
  label = label_batch[i]
  plt.axis("off")


tf.Tensor(b'./images/Img_03432.jpg', shape=(), dtype=string)
tf.Tensor(b'./images/Img_01319.jpg', shape=(), dtype=string)
tf.Tensor(b'./images/Img_01636.jpg', shape=(), dtype=string)
tf.Tensor(b'./images/Img_01429.jpg', shape=(), dtype=string)
tf.Tensor(b'./images/Img_03013.jpg', shape=(), dtype=string)


ValueError: not enough values to unpack (expected 2, got 1)

In [11]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train, x_test = x_train / 255.0, x_test / 255.0
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])
predictions = model(x_train[:1]).numpy()
tf.nn.softmax(predictions).numpy()
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
loss_fn(y_train[:1], predictions).numpy()
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])
model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test,  y_test, verbose=2)

TensorFlow version: 2.16.2


/home/archfishgame/deep/deep-learning/.venv/lib/python3.10/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.9141 - loss: 0.2973
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9578 - loss: 0.1442
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9669 - loss: 0.1084
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.9725 - loss: 0.0891
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.9760 - loss: 0.0753
313/313 - 1s - 2ms/step - accuracy: 0.9768 - loss: 0.0721


[0.07212363928556442, 0.9768000245094299]